In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '../discrete_mixflows/')
from discrete_mixflows import *
from gibbs import *

plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams["figure.figsize"]=15,7.5
plt.rcParams.update({'font.size': 24})

In [2]:
########################
########################
# target specification #
########################
########################
np.random.seed(2023)
K1=10
prbs=np.random.rand(K1)
prbs=prbs/np.sum(prbs)
def lp(x,axis=None):
    # compute the univariate log joint and conditional target pmfs
    #
    # inputs:
    #    x    : (1,d) array with state values
    #    axis : int, full conditional to calculate; returns joint if None
    # outputs:
    #   ext_lprb : if axis is None, (d,) array with log joint; else, (d,K1) array with d conditionals 
    
    ext_lprb=np.log(np.repeat(prbs[:,np.newaxis],x.shape[1],axis=1).T)
    if axis==None: return np.squeeze(ext_lprb[np.arange(0,x.shape[1]),x])
    return ext_lprb

In [4]:
########################
########################
#   Concrete approx    #
########################
########################
temperature=0.1
ref_dist=tfp.distributions.RelaxedOneHotCategorical(temperature, probs=np.ones(K1)/K1) # uniform distribution
target_dist=tfp.distributions.RelaxedOneHotCategorical(temperature, probs=prbs)

In [6]:
tfd = tfp.distributions
tfb = tfp.bijectors

nvp = tfd.TransformedDistribution(
    distribution=ref_dist,
    bijector=tfb.RealNVP(
        num_masked=6,
        shift_and_log_scale_fn=tfb.real_nvp_default_template(
            hidden_layers=[32, 32])))

In [7]:
# sampler: one-hot concrete to X
np.sum(np.round(nvp.sample())*np.arange(1,11)).astype(int)

/Users/giankdiluvi/Library/Python/3.9/lib/python/site-packages/tensorflow_probability/python/bijectors/real_nvp.py:392: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  x = tf1.layers.dense(
/Users/giankdiluvi/Library/Python/3.9/lib/python/site-packages/tensorflow_probability/python/bijectors/real_nvp.py:398: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  x = tf1.layers.dense(


3

In [8]:
from tensorflow.keras import Input
from tensorflow.keras import Model

In [9]:
from tensorflow.keras.callbacks import LambdaCallback

def train_nf(trainable_distribution, n_iters=100, n_disp=10):
    x_=tf.keras.Input(shape=(10,),dtype=tf.float32)
    log_prob_=trainable_distribution.log_prob(x_)
    model=tf.keras.Model(x_,log_prob_)
    
    model.compile(optimizer=tf.optimizers.Adam(),
                 loss=lambda _,log_prob: -log_prob)
    
    # display loss every n_disp iterations
    epoch_callback=LambdaCallBack(
        on_epoch_end=lambda n_iter, logs :
            print('\n Iteration {}/{}'.format(n_iter+1,n_iters,logs),
                  '\n\t '+(': {:.4f}, '.join(logs.keys())+': {:.4f}'.format(*logs.values())))
                  if n_iter%n_disp==0 else False
    )
    
    X_data=dist.sample(1000)
    
    history=model.fit(
        x=X_data,
        y=np.zeros((X_data.shape[0],0),dtype=np.float32),
        batch_size=X_data.shape[0],
        epochs=n_iters,
        validation_split=0,
        shuffle=True,
        verbose=False#,
        #callbacks=[epoch_callback]
    )
    return history

In [11]:
history=train_nf(nvp)

ValueError: Trying to share variable real_nvp_default_template/dense/kernel, but specified dtype float32 and found dtype float64.

originally defined at:
  File "/var/folders/k3/24b0dzl557v5m0_0658q2cxr0000gn/T/ipykernel_65722/541099623.py", line 8, in <module>
    shift_and_log_scale_fn=tfb.real_nvp_default_template(
  File "/Users/giankdiluvi/Library/Python/3.9/lib/python/site-packages/tensorflow_probability/python/bijectors/real_nvp.py", line 409, in real_nvp_default_template
    return tf1.make_template('real_nvp_default_template', _fn)
  File "/Users/giankdiluvi/Library/Python/3.9/lib/python/site-packages/tensorflow/python/ops/template.py", line 167, in make_template
    return make_template_internal(
  File "/Users/giankdiluvi/Library/Python/3.9/lib/python/site-packages/tensorflow/python/ops/template.py", line 232, in make_template_internal
    return EagerTemplate(


## ipynb implementation

In [12]:
from tensorflow.keras.layers import Layer,Dense

class NN(Layer):
    """
    Neural Network Architecture for calcualting s and t for Real-NVP
    
    :param input_shape: shape of the data coming in the layer
    :param hidden_units: Python list-like of non-negative integers, specifying the number of units in each hidden layer.
    :param activation: Activation of the hidden units
    """
    def __init__(self, input_shape, n_hidden=[512, 512], activation="relu", name="nn"):
        super(NN, self).__init__(name="nn")
        layer_list = []
        for i, hidden in enumerate(n_hidden):
            layer_list.append(Dense(hidden, activation=activation))
        self.layer_list = layer_list
        self.log_s_layer = Dense(input_shape, activation="tanh", name='log_s')
        self.t_layer = Dense(input_shape, name='t')

    def call(self, x):
        y = x
        for layer in self.layer_list:
            y = layer(y)
        log_s = self.log_s_layer(y)
        t = self.t_layer(y)
        return log_s, t

class myRealNVP(tfb.Bijector):
    """
    Implementation of a Real-NVP for Denisty Estimation. L. Dinh “Density estimation using Real NVP,” 2016.
    This implementation only works for 1D arrays.
    :param input_shape: shape of the data coming in the layer
    :param hidden_units: Python list-like of non-negative integers, specifying the number of units in each hidden layer.
    """

    def __init__(self, input_shape, n_hidden=[512, 512], forward_min_event_ndims=1, validate_args: bool = False, name="real_nvp"):
        super(myRealNVP, self).__init__(
            validate_args=validate_args, forward_min_event_ndims=forward_min_event_ndims, name=name
        )

        assert input_shape % 2 == 0
        input_shape = input_shape // 2
        nn_layer = NN(input_shape, n_hidden)
        x = tf.keras.Input(input_shape)
        log_s, t = nn_layer(x)
        self.nn = Model(x, [log_s, t], name="nn")
        
    def _bijector_fn(self, x):
        log_s, t = self.nn(x)
        return tfb.affine_scalar.AffineScalar(shift=t, log_scale=log_s)

    def _forward(self, x):
        x_a, x_b = tf.split(x, 2, axis=-1)
        y_b = x_b
        y_a = self._bijector_fn(x_b).forward(x_a)
        y = tf.concat([y_a, y_b], axis=-1)
        return y

    def _inverse(self, y):
        y_a, y_b = tf.split(y, 2, axis=-1)
        x_b = y_b
        x_a = self._bijector_fn(y_b).inverse(y_a)
        x = tf.concat([x_a, x_b], axis=-1)
        return x

    def _forward_log_det_jacobian(self, x):
        x_a, x_b = tf.split(x, 2, axis=-1)
        return self._bijector_fn(x_b).forward_log_det_jacobian(x_a, event_ndims=1)
    
    def _inverse_log_det_jacobian(self, y):
        y_a, y_b = tf.split(y, 2, axis=-1)
        return self._bijector_fn(y_b).inverse_log_det_jacobian(y_a, event_ndims=1)

In [13]:
# chain bijectors
# from https://github.com/LukasRinder/normalizing-flows/blob/master/experiments/real-nvp/real_nvp_uci.ipynb

bijectors = []
layers=2
for i in range(layers):
    bijectors.append(myRealNVP(input_shape=4))
    #bijectors.append(tfp.bijectors.Permute(permutation))

bijector = tfb.Chain(bijectors=list(reversed(bijectors)), name='chain_of_real_nvp')

flow = tfd.TransformedDistribution(
    distribution=ref_dist,
    bijector=bijector
)

In [14]:
flow.sample()

ValueError: Exception encountered when calling layer 'nn' (type NN).

Input 0 of layer "dense_4" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (5,)

Call arguments received by layer 'nn' (type NN):
  • x=tf.Tensor(shape=(5,), dtype=float32)